In [ ]:
#Improvement in progress
import numpy as np
import math
import typing
import time
import scipy
from scipy.stats import binom, multinomial, norm

# Edgeworth expansion
def edgeworth_expansion(x, n, p):
    q = 1 - p
    mu = n * p
    sigma = np.sqrt(n * p * q)
    z = (x - mu) / sigma
    skewness = (q - p) / np.sqrt(n * p * q)
    kurtosis = (1 - 6*p*q) / (n * p * q)

    phi_z = norm.pdf(z)
    Phi_z = norm.cdf(z)

    correction = (1/6) * skewness * (z**2 - 1) * phi_z + (1/24) * kurtosis * (z**3 - 3*z) * phi_z - (1/36) * skewness**2 * (z**5 - 10*z**3 + 15*z) * phi_z

    return Phi_z + correction

# P(c') with the approximation
def prob_cost_edgeworth(cost: int, num_constraints: int, prob_edge: float = 0.5) -> float:
    return edgeworth_expansion(cost, num_constraints, prob_edge)

# N(c') from paper
def number_with_cost_edgeworth_proxy(cost: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    scale = 1 << num_qubits
    return prob_cost_edgeworth(cost, num_constraints, prob_edge) * scale

# P(b, c'-b, c-b | d) from paper
def prob_common_at_distance_edgeworth(num_constraints: int, common_constraints: int, cost_1: int, cost_2: int, distance: int) -> float:
    prob_same = (math.comb(num_constraints - distance, 2) + math.comb(distance, 2)) / math.comb(num_constraints, 2)
    prob_neither = prob_same / 2
    prob_both = prob_neither
    prob_one = (1 - prob_neither - prob_both) / 2
    return multinomial.pmf(
        [common_constraints, cost_1 - common_constraints, cost_2 - common_constraints, num_constraints + common_constraints - (cost_1 + cost_2)],
        num_constraints,
        [prob_both, prob_one, prob_one, prob_neither],
    )

# N(c'; d, c) from paper
def number_of_costs_at_distance_edgeworth_proxy(cost_1: int, cost_2: int, distance: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    sum = 0
    for common_constraints in range(max(0, cost_1 + cost_2 - num_constraints), min(cost_1, cost_2) + 1):
        sum += prob_common_at_distance_edgeworth(num_constraints, common_constraints, cost_1, cost_2, distance)

    p_cost = prob_cost_edgeworth(cost_1, num_constraints, prob_edge)
    return (math.comb(num_qubits, distance) / p_cost) * sum

# Computes the sum inside the for loop of Algorithm 1 in paper
def compute_amplitude_sum_edgeworth(prev_amplitudes: np.ndarray, gamma: float, beta: float, cost_1: int, num_constraints: int, num_qubits: int) -> complex:
    sum = 0
    for cost_2 in range(num_constraints + 1):
        for distance in range(num_qubits + 1):
            beta_factor = (np.cos(beta) ** (num_qubits - distance)) * ((-1j * np.sin(beta)) ** distance)
            gamma_factor = np.exp(-1j * gamma * cost_2)
            num_costs_at_distance = number_of_costs_at_distance_edgeworth_proxy(cost_1, cost_2, distance, num_constraints, num_qubits)
            sum += beta_factor * gamma_factor * prev_amplitudes[cost_2] * num_costs_at_distance
    return sum

# Algorithm 1 from paper
def QAOA_edgeworth_proxy(p: int, gamma: np.ndarray, beta: np.ndarray, num_constraints: int, num_qubits: int):
    num_costs = num_constraints + 1
    amplitude_proxies = np.zeros([p + 1, num_costs], dtype=complex)
    init_amplitude = np.sqrt(1 / (1 << num_qubits))
    for i in range(num_costs):
        amplitude_proxies[0][i] = init_amplitude

    for current_depth in range(1, p + 1):
        for cost_1 in range(num_costs):
            amplitude_proxies[current_depth][cost_1] = compute_amplitude_sum_edgeworth(
                amplitude_proxies[current_depth - 1], gamma[current_depth - 1], beta[current_depth - 1], cost_1, num_constraints, num_qubits
            )

    expected_proxy = 0
    for cost in range(num_costs):
        expected_proxy += number_with_cost_edgeworth_proxy(cost, num_constraints, num_qubits) * (abs(amplitude_proxies[p][cost]) ** 2) * cost

    return amplitude_proxies, expected_proxy

def inverse_edgeworth_proxy_objective_function(num_constraints: int, num_qubits: int, p: int, expectations: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        _, expectation = QAOA_edgeworth_proxy(p, gamma, beta, num_constraints, num_qubits)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        return -expectation

    return inverse_objective

def QAOA_edgeworth_proxy_run(
    num_constraints: int,
    num_qubits: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    expectations: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_edgeworth_proxy_objective_function(num_constraints, num_qubits, p, expectations),
        init_freq,
        args=(),
        method=optimizer_method,
        options=optimizer_options,
    )
    end_time = time.time()

    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))

    gamma, beta = result.x[:p], result.x[p:]
    _, expectation = QAOA_edgeworth_proxy(p, gamma, beta, num_constraints, num_qubits)

    return {
        "gamma": gamma,
        "beta": beta,
        "expectation": expectation,
        "runtime": end_time - start_time,  # measured in seconds
        "num_QAOA_calls": result.nfev,  # Calls to the proxy of course
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message,
    }

In [ ]:
#Original QAOA
import typing
import qokit
import numpy as np
import scipy
import time
from qokit.fur import choose_simulator
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType

"""
This will serve as a module for QAOA simulation functionalities. 

The main function is QAOA_run, which uses QAOA with specified parameters for the ising model 
that it is passed. 

Most other functions are written only for the purpose of QAOA_run to use them. 
"""


def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None, simulator_name: str = "auto") -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = choose_simulator(name=simulator_name)
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim, simulator_name=simulator_name)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim, simulator_name=simulator_name)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_state(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"):
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_statevector(result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_expectation(result, preserve_state=True)


def get_overlap(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None, simulator_name: str = "auto"
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result, simulator_name=simulator_name)
    return simulator.get_overlap(result, preserve_state=True, optimization_type="max")


def inverse_objective_function(
    ising_model: TermsType, N: int, p: int, mixer: str, expectations: list[np.ndarray] | None, overlaps: list[np.ndarray] | None, simulator_name: str = "auto"
) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        simulator, result = get_simulator_and_result(N, ising_model, gamma, beta, simulator_name=simulator_name)
        expectation = get_expectation(N, ising_model, gamma, beta, simulator, result, simulator_name=simulator_name)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        if overlaps is not None:
            overlaps.append((current_time, get_overlap(N, ising_model, gamma, beta, simulator, result, simulator_name=simulator_name)))

        return -expectation

    return inverse_objective


def QAOA_run(
    ising_model: TermsType,
    N: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    mixer: str = "x",  # Using a different mixer is not yet supported
    expectations: list[np.ndarray] | None = None,
    overlaps: list[np.ndarray] | None = None,
    simulator_name: str = "auto",
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_objective_function(ising_model, N, p, mixer, expectations, overlaps, simulator_name=simulator_name), init_freq, args=(), method=optimizer_method, options=optimizer_options
    )
    # the above returns a scipy optimization result object that has multiple attributes
    # result.x gives the optimal solutionsol.success #bool whether algorithm succeeded
    # result.message #message of why algorithms terminated
    # result.nfev is number of iterations used (here, number of QAOA calls)
    end_time = time.time()

    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))
    
    if overlaps is not None:
        overlaps = list(map(make_time_relative, overlaps))

    gamma, beta = result.x[:p], result.x[p:]

    return {
        "gamma": gamma,
        "beta": beta,
        "state": get_state(N, ising_model, gamma, beta, simulator_name=simulator_name),
        "expectation": get_expectation(N, ising_model, gamma, beta, simulator_name=simulator_name),
        "overlap": get_overlap(N, ising_model, gamma, beta, simulator_name=simulator_name),
        "runtime": end_time - start_time,  # measured in seconds
        "num_QAOA_calls": result.nfev,
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message,
    }

In [ ]:
#QAOA_paper_proxy
import numpy as np
import math
import typing
import time
import scipy
from scipy.stats import binom, multinomial

"""
This file implements the QAOA proxy algorithm for MaxCut from:
https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.6.023171
"""


# P(c') from paper
def prob_cost_paper(cost: int, num_constraints: int, prob_edge: float = 0.5) -> float:
    return binom.pmf(cost, num_constraints, prob_edge)


# N(c') from paper
def number_with_cost_paper_proxy(cost: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    scale = 1 << num_qubits
    return prob_cost_paper(cost, num_constraints, prob_edge) * scale


# P(b, c'-b, c-b | d) from paper
def prob_common_at_distance_paper(num_constraints: int, common_constraints: int, cost_1: int, cost_2: int, distance: int) -> float:
    prob_same = (math.comb(num_constraints - distance, 2) + math.comb(distance, 2)) / math.comb(num_constraints, 2)
    prob_neither = prob_same / 2
    prob_both = prob_neither
    prob_one = (1 - prob_neither - prob_both) / 2
    return multinomial.pmf(
        [common_constraints, cost_1 - common_constraints, cost_2 - common_constraints, num_constraints + common_constraints - (cost_1 + cost_2)],
        num_constraints,
        [prob_both, prob_one, prob_one, prob_neither],
    )


# N(c'; d, c) from paper
def number_of_costs_at_distance_paper_proxy(cost_1: int, cost_2: int, distance: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    sum = 0
    for common_constraints in range(max(0, cost_1 + cost_2 - num_constraints), min(cost_1, cost_2) + 1):
        sum += prob_common_at_distance_paper(num_constraints, common_constraints, cost_1, cost_2, distance)

    p_cost = prob_cost_paper(cost_1, num_constraints, prob_edge)
    return (math.comb(num_qubits, distance) / p_cost) * sum


# Computes the sum inside the for loop of Algorithm 1 in paper
def compute_amplitude_sum_paper(prev_amplitudes: np.ndarray, gamma: float, beta: float, cost_1: int, num_constraints: int, num_qubits: int) -> complex:
    sum = 0
    for cost_2 in range(num_constraints + 1):
        for distance in range(num_qubits + 1):
            # Should I np-ify all of the stuff here?
            beta_factor = (np.cos(beta) ** (num_qubits - distance)) * ((-1j * np.sin(beta)) ** distance)
            gamma_factor = np.exp(-1j * gamma * cost_2)
            num_costs_at_distance = number_of_costs_at_distance_paper_proxy(cost_1, cost_2, distance, num_constraints, num_qubits)
            sum += beta_factor * gamma_factor * prev_amplitudes[cost_2] * num_costs_at_distance
    return sum


# TODO: What if instead of optimizing expectation proxy we instead optimize high cost amplitudes (using e.g. exponential weighting)
# Algorithm 1 from paper
# num_constraints = number of edges, and num_qubits = number of vertices
def QAOA_paper_proxy(p: int, gamma: np.ndarray, beta: np.ndarray, num_constraints: int, num_qubits: int):
    num_costs = num_constraints + 1
    amplitude_proxies = np.zeros([p + 1, num_costs], dtype=complex)
    init_amplitude = np.sqrt(1 / (1 << num_qubits))
    for i in range(num_costs):
        amplitude_proxies[0][i] = init_amplitude

    for current_depth in range(1, p + 1):
        for cost_1 in range(num_costs):
            amplitude_proxies[current_depth][cost_1] = compute_amplitude_sum_paper(
                amplitude_proxies[current_depth - 1], gamma[current_depth - 1], beta[current_depth - 1], cost_1, num_constraints, num_qubits
            )

    expected_proxy = 0
    for cost in range(num_costs):
        expected_proxy += number_with_cost_paper_proxy(cost, num_constraints, num_qubits) * (abs(amplitude_proxies[p][cost]) ** 2) * cost

    return amplitude_proxies, expected_proxy


def inverse_paper_proxy_objective_function(num_constraints: int, num_qubits: int, p: int, expectations: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        _, expectation = QAOA_paper_proxy(p, gamma, beta, num_constraints, num_qubits)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        return -expectation

    return inverse_objective


def QAOA_paper_proxy_run(
    num_constraints: int,
    num_qubits: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    expectations: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_paper_proxy_objective_function(num_constraints, num_qubits, p, expectations),
        init_freq,
        args=(),
        method=optimizer_method,
        options=optimizer_options,
    )
    # the above returns a scipy optimization result object that has multiple attributes
    # result.x gives the optimal solutionsol.success #bool whether algorithm succeeded
    # result.message #message of why algorithms terminated
    # result.nfev is number of iterations used (here, number of QAOA calls)
    end_time = time.time()

    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))

    gamma, beta = result.x[:p], result.x[p:]
    _, expectation = QAOA_paper_proxy(p, gamma, beta, num_constraints, num_qubits)

    return {
        "gamma": gamma,
        "beta": beta,
        "expectation": expectation,
        "runtime": end_time - start_time,  # measured in seconds
        "num_QAOA_calls": result.nfev,  # Calls to the proxy of course
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message,
    }

In [ ]:
import numpy as np

# テスト用のパラメータ
N = 5  # 例として5 qubits
p = 1  # 例として1レイヤー
num_constraints = 10  # 例として10 constraints
num_qubits = 5  # 例として5 qubits
test_gamma = np.array([np.pi / 4])  # テスト用のgamma
test_beta = np.array([np.pi / 4])  # テスト用のbeta

# ランダムなイジング模型
def generate_random_ising_model(N: int):
    J = []
    for i in range(N):
        for j in range(i+1, N+1):
            if i < j:
                rand_coef = np.random.uniform(-1, 1)
                indices = (i, j)
                J.append((rand_coef, indices))
    return J

ising_model = generate_random_ising_model(N)  # イジング模型

# 各関数の結果を確認
try:
    result_original = QAOA_run(ising_model, N, p, test_gamma, test_beta, expectations=[])
    print("Original QAOA result:", result_original)

    result_paper = QAOA_paper_proxy_run(num_constraints, num_qubits, p, test_gamma, test_beta, expectations=[])
    print("QAOA Paper Proxy result:", result_paper)

    result_edgeworth = QAOA_edgeworth_proxy_run(num_constraints, num_qubits, p, test_gamma, test_beta, expectations=[])
    print("QAOA Edgeworth Proxy result:", result_edgeworth)
except Exception as e:
    print("Error:", e)


In [ ]:
import numpy as np
import time

# 初期化パラメータの設定
N = 5  # 例として5 qubits
p = 1  # 例として1レイヤー
num_constraints = 10  # 例として10 constraints
num_qubits = 5  # 例として5 qubits

# 実行パラメータの範囲をさらに縮小
gamma_values = np.linspace(0, np.pi, 2)  # gammaの範囲をさらに縮小
beta_values = np.linspace(0, np.pi, 2)  # betaの範囲をさらに縮小

# 結果の保存
expectations_original = []
expectations_paper = []
expectations_edgeworth = []

# ランダムなイジング模型
def generate_random_ising_model(N: int):
    J = []
    for i in range(N):
        for j in range(i+1, N+1):
            if i < j:
                rand_coef = np.random.uniform(-1, 1)
                indices = (i, j)
                J.append((rand_coef, indices))
    return J

ising_model = generate_random_ising_model(N)  # イジング模型

# エクスペクテーションの計算
for gamma in gamma_values:
    for beta in beta_values:
        start_time = time.time()
        try:
            # 各関数を呼び出し、エクスペクテーションを計算
            result_original = QAOA_run(ising_model, N, p, np.array([gamma]), np.array([beta]), expectations=[])
            result_paper = QAOA_paper_proxy_run(num_constraints, num_qubits, p, np.array([gamma]), np.array([beta]), expectations=[])
            result_edgeworth = QAOA_edgeworth_proxy_run(num_constraints, num_qubits, p, np.array([gamma]), np.array([beta]), expectations=[])

            expectations_original.append(result_original["expectation"])
            expectations_paper.append(result_paper["expectation"])
            expectations_edgeworth.append(result_edgeworth["expectation"])

            print(f"Gamma: {gamma}, Beta: {beta}, Time: {time.time() - start_time:.2f} seconds")
        except Exception as e:
            print(f"Error at gamma={gamma}, beta={beta}: {e}")
            print(f"Gamma: {gamma}, Beta: {beta}, Time: {time.time() - start_time:.2f} seconds")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures

# 初期化パラメータの設定
N = 5  # 例として5 qubits
p = 1  # 例として1レイヤー
num_constraints = 10  # 例として10 constraints
num_qubits = 5  # 例として5 qubits

# 実行パラメータの範囲を縮小
gamma_values = np.linspace(0, np.pi, 3)  # 範囲を縮小
beta_values = np.linspace(0, np.pi, 3)  # 範囲を縮小

# 結果の保存
expectations_original = []
expectations_paper = []
expectations_edgeworth = []

# ランダムなイジング模型の生成
def generate_random_ising_model(N: int):
    J = []
    for i in range(N):
        for j in range(i+1, N+1):
            if i < j:
                rand_coef = np.random.uniform(-1, 1)
                indices = (i, j)
                J.append((rand_coef, indices))
    return J

ising_model = generate_random_ising_model(N)  # イジング模型

# 並列処理用の関数
def compute_expectations(gamma, beta):
    result_original = QAOA_run(ising_model, N, p, np.array([gamma]), np.array([beta]), expectations=[])
    result_paper = QAOA_paper_proxy_run(num_constraints, num_qubits, p, np.array([gamma]), np.array([beta]), expectations=[])
    result_edgeworth = QAOA_edgeworth_proxy_run(num_constraints, num_qubits, p, np.array([gamma]), np.array([beta]), expectations=[])
    
    return (result_original["expectation"], result_paper["expectation"], result_edgeworth["expectation"])

# 並列処理の実行
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(compute_expectations, gamma, beta) for gamma in gamma_values for beta in beta_values]
    for future in concurrent.futures.as_completed(futures):
        result_original, result_paper, result_edgeworth = future.result()
        expectations_original.append(result_original)
        expectations_paper.append(result_paper)
        expectations_edgeworth.append(result_edgeworth)

# グラフの作成
plt.figure(figsize=(10, 6))

plt.plot(expectations_original, label='Original')
plt.plot(expectations_paper, label='Paper Proxy')
plt.plot(expectations_edgeworth, label='Edgeworth Proxy')

plt.xlabel('Iteration')
plt.ylabel('Expectation Value')
plt.title('Comparison of Expectation Values')
plt.legend()
plt.show()